In [5]:
import tensorflow as tf
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from tqdm import tqdm
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import os
import importlib

dir_ml = 'C:/Users/Sten Stokroos/Desktop/NEW/zelf/Data/out'
randseed = 42
print("random seed: ", randseed)
np.random.seed(randseed)

random seed:  42


In [9]:
def choose_data(dat, test_size, val_size, fold=0, n_folds=5):
    if dat == 'ml2':
        train = pd.read_csv(os.path.join(dir_ml, 'ml_train2.csv'), sep="\t", header=None, names=['userId', 'songId', 'rating'], usecols=[0, 1, 2], engine="python")
        test = pd.read_csv(os.path.join(dir_ml, 'ml_test2.csv'), sep="\t", header=None, names=['userId', 'songId', 'rating'], usecols=[0, 1, 2], engine="python")
        val = None  # Assuming no validation set for 'ml2'
    elif dat == 'ml':
        ml_full = pd.read_csv(os.path.join(dir_ml, 'ml-1m_full.csv'), sep="\t", header=None, names=['userId', 'songId', 'rating'], usecols=[0, 1, 2], engine="python")

        user_ids = ml_full['userId'].unique()
        item_ids = ml_full['songId'].unique()

        n_users = len(user_ids)
        n_items = len(item_ids)
        
        # Split user IDs for train and test sets
        train, test = train_test_split(ml_full, test_size=test_size, random_state=42)

        # Shuffle the training set
        train = train.sample(frac=1, random_state=42).reset_index(drop=True)
        
        # Create folds for cross-validation
        fold_size = int(len(train) / n_folds)
        val = train.iloc[fold * fold_size: (fold + 1) * fold_size]
        
        # Remaining data is the training set for this fold
        train = pd.concat([train.iloc[:fold * fold_size], train.iloc[(fold + 1) * fold_size:]]).reset_index(drop=True)
    else:
        print('Wrong data input')
        return None, None, None
    # Print the sizes of the datasets
    print(f"Train set size: {train.shape[0]} ratings")
    print(f"Validation set size: {val.shape[0]} ratings")
    print(f"Test set size: {test.shape[0]} ratings")

    return train, val, test, n_users, n_items

def load_confounders(dat, k):
    CAUSEFIT_DIR = f'C:/Users/Sten Stokroos/Desktop/NEW/zelf/Data/exposure_output/{dat}_exp_k_{k}.csv'
    conf_df = pd.read_csv(CAUSEFIT_DIR, header=None)
    confounder_data = conf_df.to_numpy().T
    return confounder_data

def load_data_rating(dat, columns=[0, 1, 2], sep="\t", include_validation=False, test_size=0.1, val_size=0.1, fold=0, n_folds=5):
    train, val, test, n_users, n_items = choose_data(dat, test_size, val_size, fold=fold, n_folds=n_folds)

    if train is None or test is None:
        return None, None, None, None, None

    def build_matrix(df):
        rows = []
        cols = []
        ratings = []
        for line in df.itertuples():
            rows.append(line[1])
            cols.append(line[2])
            ratings.append(line[3])
        return csr_matrix((ratings, (rows, cols)), shape=(n_users, n_items)).todok()

    train_matrix = build_matrix(train)
    test_matrix = build_matrix(test)
    vad_matrix = build_matrix(val) if val is not None else None

    print("Load data finished. Number of users:", n_users, "Number of items:", n_items)
    return train_matrix, test_matrix, vad_matrix, n_users, n_items


In [10]:
def run_model(module_name, class_name, k, dat='ml', include_validation=False, use_confounder=False, use_exposure=False, test_size=0.1, val_size=0.1, hidden_neuron=500, learning_rate=0.001, reg_rate=0.1, epoch=20, batch_size=200, verbose=False, T=1, display_step=1000, save_path=None, fold=0, n_folds=5):
    train, test, vad, user, item = load_data_rating(dat, columns=[0, 1, 2], sep="\t", include_validation=include_validation, test_size=test_size, val_size=val_size, fold=fold, n_folds=n_folds)

    confounder_data = None
    exposure_data = None

    if use_confounder:
        confounder_data = load_confounders(dat, k)
    if use_exposure:
        exposure_data = (train > 0).astype(np.float32).todense().T

    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True

    with tf.compat.v1.Session(config=config) as sess:
        module = importlib.import_module(module_name)
        model_class = getattr(module, class_name)
        final_model = model_class(sess, user, item, learning_rate=learning_rate, reg_rate=reg_rate, epoch=epoch, batch_size=batch_size, verbose=verbose, T=T, display_step=display_step)

        final_model.build_network(hidden_neuron=hidden_neuron)
        
        if confounder_data is not None and exposure_data is not None:
            final_model.execute(train, vad, confounder_data, exposure_data)
        elif confounder_data is not None:
            final_model.execute(train, vad, confounder_data)
        else:
            final_model.execute(train, vad)

        if vad is not None:
            if confounder_data is not None and exposure_data is not None:
                rmse, mae = final_model.test(vad, confounder_data, exposure_data)
            elif confounder_data is not None:
                rmse, mae = final_model.test(vad, confounder_data)
            else:
                rmse, mae = final_model.test(vad)
        else:
            rmse, mae = None, None

    return rmse, mae


In [11]:
def objective_urec2confexp(params):
    learning_rate = params['learning_rate']
    reg_rate = params['reg_rate']
    hidden_neuron = params['hidden_neuron']
    k = params['k']

    n_folds = 5
    rmse_list = []
    for fold in range(n_folds):
        rmse, mae = run_model('urec_2_confexp', 'UAutoRec2confexp', k, dat='ml', include_validation=True, use_confounder=True, use_exposure=True, hidden_neuron=hidden_neuron, learning_rate=learning_rate, reg_rate=reg_rate, batch_size=500, epoch=30, fold=fold, n_folds=n_folds)
        rmse_list.append(rmse)
    
    avg_rmse = np.mean(rmse_list)
    return {'loss': avg_rmse, 'status': STATUS_OK}


space = {
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.1)),
    'reg_rate': hp.loguniform('reg_rate', np.log(0.001), np.log(1000)),
    'hidden_neuron': hp.choice('hidden_neuron', [10, 20, 40, 80, 100, 200, 300, 400, 500]),
    'k': hp.choice('k', [1, 2, 5, 10, 20, 32, 50, 100]),
}

trials_urec2confexp = Trials()
best_urec2confexp = fmin(fn=objective_urec2confexp, space=space, algo=tpe.suggest, max_evals=10, trials=trials_urec2confexp)

print("Best parameters for UAutoRec1conf:", best_urec2confexp)

best_params_urec2confexp = {
    'learning_rate': best_urec2confexp['learning_rate'],
    'reg_rate': best_urec2confexp['reg_rate'],
    'hidden_neuron': [10, 20, 40, 80, 100, 200, 300, 400, 500][best_urec2confexp['hidden_neuron']],
    'k': [1, 2, 5, 10, 20, 32, 50, 100][best_urec2confexp['k']]
}

print("Best parameters for UAutoRec1conf in detailed form:", best_params_urec2confexp)


Train set size: 720153 ratings                        
Validation set size: 180038 ratings                   
Test set size: 100022 ratings                         
Load data finished. Number of users:                  
6040                                                  
Number of items:                                      
3706                                                  
UAutoRec with Confounder and Exposure.                
  0%|          | 0/10 [00:12<?, ?trial/s, best loss=?]

Training: 100%|##########| 30/30 [03:05<00:00,  6.20s/epoch, Loss=7.97e+4, RMSE=1, MAE=0.801]


Train set size: 720153 ratings                        
Validation set size: 180038 ratings                   
Test set size: 100022 ratings                         
Load data finished. Number of users:                  
6040                                                  
Number of items:                                      
3706                                                  
UAutoRec with Confounder and Exposure.                
  0%|          | 0/10 [05:23<?, ?trial/s, best loss=?]

Training: 100%|##########| 30/30 [03:03<00:00,  6.13s/epoch, Loss=8.08e+4, RMSE=1.01, MAE=0.802]


Train set size: 720153 ratings                        
Validation set size: 180038 ratings                   
Test set size: 100022 ratings                         
Load data finished. Number of users:                  
6040                                                  
Number of items:                                      
3706                                                  
UAutoRec with Confounder and Exposure.                
  0%|          | 0/10 [10:30<?, ?trial/s, best loss=?]

...

Train set size: 720153 ratings                                                      
Validation set size: 180038 ratings                                                 
Test set size: 100022 ratings                                                       
Load data finished. Number of users:                                                
6040                                                                                
Number of items:                                                                    
3706                                                                                
UAutoRec with Confounder and Exposure.                                              
 90%|█████████ | 9/10 [4:12:31<26:32, 1592.24s/trial, best loss: 0.9414419705683805]

Training: 100%|##########| 30/30 [03:14<00:00,  6.47s/epoch, Loss=5.85e+4, RMSE=0.991, MAE=0.791]


Train set size: 720153 ratings                                                      
Validation set size: 180038 ratings                                                 
Test set size: 100022 ratings                                                       
Load data finished. Number of users:                                                
6040                                                                                
Number of items:                                                                    
3706                                                                                
UAutoRec with Confounder and Exposure.                                              
 90%|█████████ | 9/10 [4:17:58<26:32, 1592.24s/trial, best loss: 0.9414419705683805]

Training: 100%|##########| 30/30 [03:10<00:00,  6.34s/epoch, Loss=5.89e+4, RMSE=0.988, MAE=0.79]


100%|██████████| 10/10 [4:23:05<00:00, 1578.58s/trial, best loss: 0.9414419705683805]
Best parameters for UAutoRec1conf: {'hidden_neuron': 4, 'k': 1, 'learning_rate': 0.007367930080706986, 'reg_rate': 0.04815375606711562}
Best parameters for UAutoRec1conf in detailed form: {'learning_rate': 0.007367930080706986, 'reg_rate': 0.04815375606711562, 'hidden_neuron': 100, 'k': 2}


In [13]:
def run_model(module_name, class_name, k, dat='ml', include_validation=False, use_confounder=False, use_exposure=False, test_size=0.1, val_size=0.1, hidden_neuron=500, learning_rate=0.001, reg_rate=0.1, epoch=20, batch_size=200, verbose=False, T=1, display_step=1000, save_path=None):
    train, test, vad, user, item = load_data_rating(dat, columns=[0, 1, 2], sep="\t", include_validation=include_validation, test_size=test_size, val_size=val_size)

    confounder_data = None
    exposure_data = None

    if use_confounder:
        # Load confounder data
        confounder_data = load_confounders(dat, k)
    if use_exposure:
        # Create exposure matrix
        exposure_data = (train > 0).astype(np.float32).todense().T
    
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True

    with tf.compat.v1.Session(config=config) as sess:
        # Dynamically import the module and create an instance of the chosen class
        module = importlib.import_module(module_name)
        model_class = getattr(module, class_name)
        final_model = model_class(sess, user, item, learning_rate=learning_rate, reg_rate=reg_rate, epoch=epoch, batch_size=batch_size, verbose=verbose, T=T, display_step=display_step)

        final_model.build_network(hidden_neuron=hidden_neuron)
        
        # Determine how many arguments to pass to the execute function
        if confounder_data is not None and exposure_data is not None:
            final_model.execute(train, test, confounder_data, exposure_data)
        elif confounder_data is not None:
            final_model.execute(train, test, confounder_data)
        else:
            final_model.execute(train, test)

        # Save the training and test RMSE values
        if save_path:
            class_folder = os.path.join(save_path, class_name.lower())
            os.makedirs(class_folder, exist_ok=True)

            train_loss_file = os.path.join(class_folder, f'TUNEDLONG_train_loss_highepochs2_{dat}_k{k}.npy')
            test_rmse_file = os.path.join(class_folder, f'TUNEDLONG_test_rmsehighepochs2_{dat}_k{k}.npy')


            np.save(train_loss_file, np.array(final_model.train_loss_history))
            np.save(test_rmse_file, np.array(final_model.test_rmse_history))



# Best parameters for UAutoRec2confexp in detailed form: {'learning_rate': 0.007367930080706986, 'reg_rate': 0.04815375606711562, 'hidden_neuron': 100, 'k': 2} THIS IS WITH 5 FOLDS

# print("Best parameters for UAutoRec1conf in detailed form:", best_params_urec1conf)
result_path = 'C:/Users/Sten Stokroos/Desktop/NEW/zelf/results'
run_model('urec_2_confexp', 'UAutoRec2confexp', best_params_urec2confexp['k'], dat='ml', include_validation=False, use_confounder=True, use_exposure=True, 
          hidden_neuron= best_params_urec2confexp['hidden_neuron'], 
          learning_rate=best_params_urec2confexp['learning_rate'], 
          reg_rate=best_params_urec2confexp['reg_rate'], 
          batch_size=500, epoch=100, save_path = result_path)


Train set size: 720153 ratings
Validation set size: 180038 ratings
Test set size: 100022 ratings
Load data finished. Number of users: 6040 Number of items: 3706
UAutoRec with Confounder and Exposure.


Training: 100%|██████████| 100/100 [07:05<00:00,  4.26s/epoch, Loss=3.62e+4, RMSE=0.928, MAE=0.73]
